<a href="https://colab.research.google.com/github/609harsh/BTP/blob/main/SQLIADS_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os

In [8]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'

In [9]:
!kaggle datasets download -d gambleryu/biggest-sql-injection-dataset -f clean_sql_dataset.csv

 41% 9.00M/21.9M [00:00<00:00, 25.0MB/s]
100% 21.9M/21.9M [00:00<00:00, 53.8MB/s]


In [10]:
import pandas as pd
sql_dataset = 'clean_sql_dataset.csv.zip'

In [11]:
dataset = pd.read_csv(sql_dataset)
dataset

,Query,Label
0,""" or pg_sleep ( __TIME__ ) --",1
1,create user name identified by pass123 tempora...,1
2,AND 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1
...,...,...
148321,5555555555555555555555555555555555555555555555...,0
148322,0521094088709893969415119606172279713007042396...,0
148323,,0
148324,"""or 1=1",1


# Extracting the features for ANN Model

1. Feature: LP (Length of Query)
2. Feature: NK (Number of Keywords in Query)
3. Feature: KWS (Sum of Keyword Weights in Payload)
4. Feature: NSPA (Number of Spaces in Payload)
5. Feature: RSPA (Ratio of Spaces in Payload)
6. Feature: NSPE (Number of Special Characters in Payload)
7. Feature: RSPE (Ratio of Special Characters in Payload)
8. Feature: ROC (Ratio of Other Characters)

## Feature 1 - LP

In [12]:
def LP(query):
    return len(query)

dataset['LP'] = dataset['Query'].apply(LP)

## Feature 2 : NK - Number of Keywords

keywords:  select, union, count, group by, order by, insert, delete, update, drop table, truncate table,and, or, where, from, into, exec, xp_cmdshell, master, net,load_file, outfile, dumpfile

In [13]:
# list of SQL injection keywords
keywords = ["select", "union", "count", "group by", "order by", "insert", "delete", "update", "drop table", "truncate table",
            "and", "or", "where", "from", "into", "exec", "xp_cmdshell", "master", "net", "load_file", "outfile", "dumpfile"]

# Define a function to calculate the NK feature
def NK(query):
    return sum(query.count(keyword) for keyword in keywords)

# Add the "NK" feature to your dataset
dataset['NK'] = dataset['Query'].apply(NK)

In [14]:
dataset

,Query,Label,LP,NK
0,""" or pg_sleep ( __TIME__ ) --",1,33,1
1,create user name identified by pass123 tempora...,1,90,1
2,AND 1 = utl_inaddr.get_host_address ( ...,1,218,0
3,select * from users where id = '1' or @ @1 ...,1,90,6
4,"select * from users where id = 1 or 1#"" ( ...",1,85,6
...,...,...,...,...
148321,5555555555555555555555555555555555555555555555...,0,960,0
148322,0521094088709893969415119606172279713007042396...,0,653,0
148323,,0,5,0
148324,"""or 1=1",1,7,1


## Feature 3 : KWS (Sum of Keyword Weights in Query)
Weight -> Keyword
- 5 -> union, truncate, xp_cmdshell, load_file, outfile, dumpfile, exec
- 3 -> select, update, insert, delete, count, where, group, order, drop,
table, master, net
- 1 -> and, or, by, from, into

In [15]:
# Define the list of SQL injection keywords and their corresponding weights
keyword_weights = {
    "union": 5, "truncate": 5, "xp_cmdshell": 5, "load_file": 5, "outfile": 5, "dumpfile": 5, "exec": 5,
    "select": 3, "update": 3, "insert": 3, "delete": 3, "count": 3, "where": 3, "group": 3, "order": 3, "by": 3,
    "drop": 3, "table": 3, "master": 3, "net": 3,
    "and": 1, "or": 1, "from": 1, "into": 1
}

# Define a function to calculate the KWS feature
def KWS(query):
    return sum(keyword_weights[keyword] * query.count(keyword) for keyword in keyword_weights)

dataset['KWS'] = dataset['Query'].apply(KWS)

## Feature 4: NSPA (Number of Spaces)

In [16]:
# Define a function to calculate the NSPA feature
def NSPA(query):
    return query.count(' ') + query.count('%20')


# Add the "NSPA" feature to your dataset
dataset['NSPA'] = dataset['Query'].apply(NSPA)

In [17]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148326 entries, 0 to 148325
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Query   148326 non-null  object
 1   Label   148326 non-null  int64 
 2   LP      148326 non-null  int64 
 3   NK      148326 non-null  int64 
 4   KWS     148326 non-null  int64 
 5   NSPA    148326 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 6.8+ MB


## Feature 5: RSPA (Ratio of Spaces in Payload)

In [18]:
# Define a function to calculate the RSPA feature
def RSPA(NSPA, LP):
    return NSPA / LP

# Add the "RSPA" feature to your dataset
dataset['RSPA'] = dataset.apply(lambda row: RSPA(row['NSPA'], row['LP']), axis=1)

## Feature 6: NSPE (Number of Special Characters in Payload)

In [19]:
# Define the list of special characters
special_characters = ['+', '-', '*', '/', '=', '!=', '^=', '<>', '>=', '<=', '--', '#', '/*', '*/', "''", "'''", '@', '\\', '()']

# Define a function to calculate the NSPE feature
def NSPE(query):
    return sum(query.count(char) for char in special_characters)

# Add the "NSPE" feature to your dataset
dataset['NSPE'] = dataset['Query'].apply(NSPE)

## Feature 7: RSPE (Ratio of Special Characters in Payload)

In [20]:
# Define a function to calculate the RSPE feature
def RSPE(NSPE, LP):
    return NSPE / LP

# Add the "RSPE" feature to your dataset
dataset['RSPE'] = dataset.apply(lambda row: RSPE(row['NSPE'], row['LP']), axis=1)

## Feature 8: ROC (Ratio of Other Characters)

In [21]:
# Define a function to calculate the ROC feature
def ROC(PSNA, PSNE):
    return 1 - PSNA - PSNE

# Add the "ROC" feature to your dataset
dataset['ROC'] = dataset.apply(lambda row: ROC(row['NSPA'] / row['LP'], row['NSPE'] / row['LP']), axis=1)

# Working on the new Model

In [22]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148326 entries, 0 to 148325
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Query   148326 non-null  object 
 1   Label   148326 non-null  int64  
 2   LP      148326 non-null  int64  
 3   NK      148326 non-null  int64  
 4   KWS     148326 non-null  int64  
 5   NSPA    148326 non-null  int64  
 6   RSPA    148326 non-null  float64
 7   NSPE    148326 non-null  int64  
 8   RSPE    148326 non-null  float64
 9   ROC     148326 non-null  float64
dtypes: float64(3), int64(6), object(1)
memory usage: 11.3+ MB


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [24]:
X = dataset[['LP', 'NK', 'KWS', 'NSPA', 'RSPA', 'NSPE', 'RSPE', 'ROC']]
y = dataset['Label']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [31]:
len(X_test)

44498

In [26]:
# Standardize features (optional but often helpful)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Creating a MLP for the data

### Hidden layers: 100,50

In [27]:
%%time
# Create and train the MLP model
mlp = MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', random_state=42)
mlp.fit(X_train, y_train)

CPU times: user 2min 33s, sys: 1min 20s, total: 3min 54s
Wall time: 2min 9s


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(100, 50), random_state=42)

In [28]:
# Make predictions on the test set
y_pred = mlp.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the model performance metrics
print(f'Accuracy: {accuracy}')
print('Classification Report:\n', report)

Accuracy: 0.955031686817385
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.97      0.95     21118
           1       0.97      0.94      0.96     23380

    accuracy                           0.96     44498
   macro avg       0.95      0.96      0.95     44498
weighted avg       0.96      0.96      0.96     44498



In [33]:
#confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred,normalize="true")
print("Confusion Matrix:")
print(confusion_mat)

Confusion Matrix:
[[0.96718439 0.03281561]
 [0.05594525 0.94405475]]


### Trying other number of hidden layers

In [ ]:
%%time
# Create and train the MLP model with more layers
mlp = MLPClassifier(hidden_layer_sizes=(400, 100), activation='relu', random_state=42)
mlp.fit(X_train, y_train)

CPU times: user 42min 39s, sys: 12min 20s, total: 54min 59s
Wall time: 31min 22s


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(400, 100), random_state=42)

In [ ]:
# Make predictions on the test set
y_pred = mlp.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the model performance metrics
print(f'Accuracy: {accuracy}')
print('Classification Report:\n', report)

Accuracy: 0.9559306036226347
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.97      0.95     21118
           1       0.97      0.94      0.96     23380

    accuracy                           0.96     44498
   macro avg       0.96      0.96      0.96     44498
weighted avg       0.96      0.96      0.96     44498



In [ ]:
%%time
# Create and train the MLP model with more layers
mlp = MLPClassifier(hidden_layer_sizes=(500, 200), activation='relu', random_state=42)
mlp.fit(X_train, y_train)

CPU times: user 1h 50min 33s, sys: 23min 1s, total: 2h 13min 35s
Wall time: 1h 12min 44s


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(500, 200), random_state=42)

In [ ]:
# Make predictions on the test set
y_pred = mlp.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the model performance metrics
print(f'Accuracy: {accuracy}')
print('Classification Report:\n', report)

Accuracy: 0.9483347566182749
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.95      0.95     21118
           1       0.95      0.95      0.95     23380

    accuracy                           0.95     44498
   macro avg       0.95      0.95      0.95     44498
weighted avg       0.95      0.95      0.95     44498

